In [1]:
import pandas as pd

In [16]:
import pandas as pd

def dataloader(data):
    data = data.iloc[13:, 1:-1]
    data.columns = ["Station", "Offset", "Elevation"]
    out_of_range = data[data["Station"] == "Out of range"].index
    data.drop(out_of_range, inplace=True)
    data.dropna(inplace=True)
    data["Station_new"] = data["Station"].apply(lambda x: float("".join(str(x).split("+"))))
    data["Offset_new"] = data["Offset"].apply(lambda x: float(str(x)[:-1]))
    data["Elevation_new"] = data["Elevation"].apply(lambda x: float("".join(str(x)[:-1].split(","))))
    data = data.sort_values(by="Station_new")
    data = data.iloc[:, -3:]
    data.columns = ["Station", "Offset", "Elevation"]
    data.index = range(len(data.index))
    return data, out_of_range

def special_sort(input_data, initial_point, final_point, space, tolerance):
    n = int(((final_point - initial_point)/space)+1)
    classes = []
    init_s = initial_point

    for _ in range(n):
        s = []
        not_ok = 0
        find = False
        for i in range(len(input_data["Station"])):
            if abs(input_data["Station"][i] - init_s) <= tolerance:
                s.append(i)
                not_ok = 0
                find = True
            else:
                not_ok += 1
            
            if not_ok == 10 and find == True:
                break
        classes.append(s)
        init_s = init_s + space

    return classes

def make_classes(data, initial_point, space, error):
    sorted_classes = []
    for c in range(len(classes)):
        station_new = [initial_point for _ in range(len(classes[c]))]
        need_sort = data.loc[classes[c]]["Offset"]
        additional = data.loc[classes[c]]["Elevation"]
        need_sort = list(zip(station_new, classes[c], need_sort, additional))
        if need_sort:
            done = False
            choices = []
            for i in range(len(need_sort)):
                if abs(need_sort[i][2]) <= error:
                    choices.append([i, need_sort[i][2], need_sort[i][3]])
                    done = True
            if done:
                temp = min(choices, key=lambda x: x[1])
                exact = (need_sort[0][0], 0, 0, temp[2])
                need_sort.pop(temp[0])
            if not done:
                temp1 = float("inf")
                temp2 = float("inf")
                for i in range(len(need_sort)):
                    o = need_sort[i][2]
                    if o > 0:
                        if o < temp1:
                            temp1 = o
                            temp1i = i
                    elif o < 0:
                        o = -o
                        if o < temp2:
                            temp2 = o
                            temp2i = i
                x1 = temp2
                x2 = temp1
                y1 = need_sort[temp2i][3]
                y2 = need_sort[temp1i][3]
                exact = (need_sort[0][0], 0, 0, (x2*y1+x1*y2)/(x2+x1))
            need_sort.append(exact)
            sorted_elemnets = sorted(list(need_sort), key=lambda x: x[2])
            sorted_classes.append(sorted_elemnets)
        initial_point += space

    return sorted_classes

def final_preparation(final_classes):
    final_result = []
    for list_ in final_classes:
        final_result.append(["chainage", list_[0][0]])
        
        for item in list_:
            final_result.append([item[2], item[3]])

    result = pd.DataFrame(final_result)
    return result


def second_special_sort(input_data, initial_point, final_point, space, tolerance, pre_classes):
    n = int(((final_point - initial_point)/space)+1)
    classes = special_sort(input_data, initial_point, final_point, space, tolerance)
    visited_items = [item for sublist in classes for item in sublist]
    visited_items = set(visited_items)

    total_items = [item for sublist in pre_classes for item in sublist]
    total_items = set(total_items)
    diffrences = list(total_items - visited_items)
    
    result_for_station = [(i+1, input_data["Station"][i]) for i in diffrences]
    result_for_station_sorted = sorted(result_for_station, key= lambda x: x[1])

    return result_for_station_sorted

############################################################################################################

address = "Raw-Data/D10 points.csv"
address_result_txt = "Raw-Data/result.txt"
address_result_excel = "Raw-Data/result.csv"
diffrences_result_txt = "Raw-Data/diffrences_result.txt"
out_of_range_txt = "Raw-Data/out_of_ranges.txt"
data = pd.read_csv(address)

initial_point = 3600
final_point = 20900
space_between = 20
first_tolerance = space_between/2
second_tolerance = space_between/4
error = 0.2

############################################################################################################

prepared_data, out_of_range_index = dataloader(data)
classes = special_sort(prepared_data, initial_point, final_point, space_between, first_tolerance)
final_data = make_classes(prepared_data, initial_point, space_between, error)
result = final_preparation(final_data)

result_for_diffrences = second_special_sort(prepared_data, initial_point, final_point, space_between, second_tolerance, classes)
final_data = make_classes(prepared_data, initial_point, space_between, error)
result = final_preparation(final_data)

############################################################################################################

result.to_csv(address_result_excel, index=False, header=False)
with open(address_result_txt, "w") as file:
    for index, row in result.iterrows():
        file.write(f"{row[0]}\t{row[1]}\n")

with open(diffrences_result_txt, "w") as file:
    for row in result_for_diffrences:
        file.write(f"{row[0]}\t{row[1]}\n")

with open(out_of_range_txt, "w") as file:
    for row in out_of_range_index:
        file.write(f"{row}\n")

In [187]:
from copy import deepcopy
import pandas as pd

def dataloader(data_ad):
    data = pd.read_csv(data_ad)
    data = data.iloc[13:, 1:-1]
    data.columns = ["Station", "Offset", "Elevation"]
    out_of_range = data[data["Station"] == "Out of range"].index
    data.drop(out_of_range, inplace=True)
    data.dropna(inplace=True)
    data["Station_new"] = data["Station"].apply(lambda x: float("".join(str(x).split("+"))))
    data["Offset_new"] = data["Offset"].apply(lambda x: float(str(x)[:-1]))
    data["Elevation_new"] = data["Elevation"].apply(lambda x: float("".join(str(x)[:-1].split(","))))
    data = data.sort_values(by="Station_new")
    data = data.iloc[:, -3:]
    data.columns = ["Station", "Offset", "Elevation"]
    data.index = range(len(data.index))
    return data, out_of_range

def make_class(input_data: pd.DataFrame, station_i: int, tolerance:float) -> list:
    s = []
    not_ok = 0
    find = False
    start_temp = station_i-100
    end_temp = station_i+100
    start_temp = start_temp if start_temp>100 else 0
    end_temp = end_temp if end_temp<len(input_data["Station"])-100 else len(input_data["Station"])
    for j in range(start_temp, end_temp):
        if abs(input_data["Station"][j] - input_data["Station"][station_i]) <= tolerance:
            s.append(j)
            not_ok = 0
            find = True
        else:
            not_ok += 1
        
        if not_ok == 1 and find == True:
            break
    s.append(str(station_i)+"-id")
    return s

def add_between(data, ind, data_ind, remove_original=False, by_id=False):
    if by_id:
        ind = ind-data[0]
    first = data[:ind+1]
    last = data[ind+1:]
    first.append(data_ind)
    total = first+last
    if remove_original:
        total.pop(ind)
    return total

def sortit(classes):
    unique_tuples = {tuple(sublist) for sublist in classes}
    unique_list_of_lists = [list(t) for t in unique_tuples]
    new_unique = sorted(unique_list_of_lists, key= lambda x: x[0])
    return new_unique

def special_sort(input_data, tolerance, station_offset_allowable, number_of_each_station):
    classes = []
    stations = []

    for i in range(len(input_data["Station"])):
        if abs(input_data["Offset"][i]) <= station_offset_allowable:
            # print(input_data["Offset"][i])
            stations.append([i, input_data["Station"][i], input_data["Offset"][i]])
    
    for i in range(len(stations)):
        s_class = make_class(input_data, stations[i][0], tolerance)
        classes.append(s_class)

    need_check = True
    dev = 1
    while need_check:
        dev += 1
        need_check = False
        classes_copy = deepcopy(classes)
        for station_i in range(len(stations)):
            if len(classes_copy[station_i]) > number_of_each_station:
                start_temp, end_temp = classes[station_i][0], classes[station_i][-2]+1 # the last number is station_id
                for i in range(start_temp, end_temp):
                    if abs(input_data["Offset"][i]) <= station_offset_allowable:
                        s_class = make_class(input_data, stations[-1][0], tolerance/dev)
                        stations = add_between(stations, station_i, [i, input_data["Station"][i], input_data["Offset"][i]])
                        classes = add_between(classes, station_i, s_class)
                classes.pop(station_i)
                stations.pop(station_i)
                need_check = True
    classes = sortit(classes)

    return classes

def correct_offset(input_data, data, station_id):
    station_metric = input_data["Station"][station_id]
    if station_metric%10 == 0:
        pass
    elif int(station_metric%10) <= 1:
        station_metric = int(str(station_metric)[:-3]) 
    if int(station_metric%10) >= 9:
        station_metric = int(str(station_metric)[:-3])+1
    station_new = [station_metric for _ in range(len(data))]
    offset = list(input_data.loc[data]["Offset"])
    offset = add_between(offset, station_id-data[0], 0, remove_original=True)
    elevation = input_data.loc[data]["Elevation"]
    merged_data = list(zip(station_new, data, offset, elevation))
    return sorted(merged_data, key= lambda x: x[2])

def final_preparation(final_classes):
    final_result = []
    for list_ in final_classes:
        final_result.append(["chainage", list_[0][0]])
        for item in list_:
            final_result.append([item[2], item[3]])

    result = pd.DataFrame(final_result)
    return result

def export_data(ad1, ad2, ad3, result, out_of_range_index):
    result.to_csv(ad1, index=False, header=False)
    with open(ad2, "w") as file:
        for index, row in result.iterrows():
            file.write(f"{row[0]}\t{row[1]}\n")

    with open(ad3, "w") as file:
        for row in out_of_range_index:
            file.write(f"{row}\n")

def main(ad, tolerance, station_offset_allowable, max_number_of_each):
    prepared_data, out_of_range_index = dataloader(ad)
    classes = special_sort(prepared_data, tolerance, station_offset_allowable, max_number_of_each)
    corrected_classes = []
    for i in range(len(classes)):
        item = correct_offset(prepared_data, classes[i][:-1], int(classes[i][-1][:-3]))
        corrected_classes.append(item)
    result = final_preparation(corrected_classes)
    return result, out_of_range_index


address = "Raw-Data/D10 points.csv"
address_result_txt = "result.txt"
address_result_excel = "result.csv"
out_of_range_txt = "out_of_ranges.txt"

tolerance = 2
station_offset_allowable = 0.1
max_number_of_each = 30

result, out_of_range_index = main(address, tolerance, station_offset_allowable, max_number_of_each)
export_data(address_result_excel, address_result_txt, out_of_range_txt, result, out_of_range_index)



In [155]:
result

[[(20899.92, 14912, -14.974, 1304.466),
  (20899.92, 14929, -14.92, 1307.202),
  (20899.92, 14922, -14.903, 1305.993),
  (20899.92, 14910, -13.897, 1304.482),
  (20899.92, 14905, -10.302, 1305.045),
  (20899.92, 14918, -9.842, 1306.246),
  (20899.92, 14909, -9.464, 1305.014),
  (20899.92, 14906, -5.398, 1305.162),
  (20899.92, 14927, -3.263, 1307.508),
  (20899.92, 14925, -0.001, 1307.561),
  (20899.92, 14915, 0.011, 1306.403),
  (20899.92, 14907, 0.064, 1305.168),
  (20899.92, 14921, 1.598, 1306.389),
  (20899.92, 14908, 3.097, 1305.162),
  (20899.92, 14917, 4.559, 1306.421),
  (20899.92, 14928, 4.585, 1308.857),
  (20899.92, 14913, 6.217, 1305.251),
  (20899.92, 14916, 7.506, 1307.62),
  (20899.92, 14923, 8.28, 1307.421),
  (20899.92, 14911, 10.812, 1305.686),
  (20899.92, 14919, 11.569, 1306.321),
  (20899.92, 14914, 13.549, 1305.031),
  (20899.92, 14920, 13.619, 1306.025),
  (20899.92, 14924, 14.082, 1306.942),
  (20899.92, 14926, 15.245, 1306.946)],
 [(20899.92, 14929, -14.92, 130

In [123]:
unique_tuples = {tuple(sublist) for sublist in classes}
unique_list_of_lists = [list(t) for t in unique_tuples]
new_unique = sorted(unique_list_of_lists, key= lambda x: x[0])
new_unique

[[0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22],
 [23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40],
 [41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58],
 [59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82],
 [83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104],
 [105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124],
 [125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141],
 [142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,
  156,
  157,
  158,
  159],
 [160,
  161,
  162,
  163,
  164,
  165,
  166,
 

In [124]:
a = 0
for i in new_unique:
    for j in i:
        a+=1

a

8205

In [224]:
from copy import deepcopy
import pandas as pd
from tqdm.auto import tqdm

def dataloader(data_ad):
    data = pd.read_csv(data_ad)
    data = data.iloc[13:, 1:-1]
    data.columns = ["Station", "Offset", "Elevation"]
    out_of_range = data[data["Station"] == "Out of range"].index
    data.drop(out_of_range, inplace=True)
    data.dropna(inplace=True)
    data["Station_new"] = data["Station"].apply(lambda x: float("".join(str(x).split("+"))))
    data["Offset_new"] = data["Offset"].apply(lambda x: float(str(x)[:-1]))
    data["Elevation_new"] = data["Elevation"].apply(lambda x: float("".join(str(x)[:-1].split(","))))
    data = data.sort_values(by="Station_new")
    data = data.iloc[:, -3:]
    data.columns = ["Station", "Offset", "Elevation"]
    data.index = range(len(data.index))
    return data, out_of_range

def make_class(input_data: pd.DataFrame, station_i: int, tolerance:list) -> list:
    s = []
    not_ok = 0
    find = False
    start_temp = station_i-100
    end_temp = station_i+100
    start_temp = start_temp if start_temp>100 else 0
    end_temp = end_temp if end_temp<len(input_data["Station"])-100 else len(input_data["Station"])
    for j in range(start_temp, end_temp):
        temp = input_data["Station"][j] - input_data["Station"][station_i]
        if temp >= 0  and temp <= tolerance[0]:
            s.append(j)
            not_ok = 0
            find = True
        elif temp <= 0 and temp >= -tolerance[1]:
            s.append(j)
            not_ok = 0
            find = True
        else:
            not_ok += 1
        
        if not_ok == 1 and find == True:
            break
    s.append(str(station_i)+"-id")
    return s

def add_between(data, ind, data_ind, remove_original=False, by_id=False):
    if by_id:
        ind = ind-data[0]
    first = data[:ind+1]
    last = data[ind+1:]
    first.append(data_ind)
    total = first+last
    if remove_original:
        total.pop(ind)
    return total

def sortit(classes):
    unique_tuples = {tuple(sublist) for sublist in classes}
    unique_list_of_lists = [list(t) for t in unique_tuples]
    new_unique = sorted(unique_list_of_lists, key= lambda x: x[0])
    return new_unique


def tolerance_prepare(i, stations, dev=2):
    if i == 0: 
        tolerance = ((stations[i+1][1] - stations[i][1])/dev, stations[i][1]/dev)
    elif i == len(stations)-1:
        tolerance = (stations[i][1]/dev, (stations[i][1] - stations[i-1][1])/dev)
    else:
        tolerance = ((stations[i+1][1] - stations[i][1])/dev, (stations[i][1] - stations[i-1][1])/dev)
    return tolerance

def special_sort(input_data, station_offset_allowable, number_of_each_station):
    classes = []
    stations = []

    for i in range(len(input_data["Station"])):
        # print(abs(input_data["Offset"][i]), station_offset_allowable)
        if abs(input_data["Offset"][i]) <= station_offset_allowable:
            # print(abs(input_data["Offset"][i]), station_offset_allowable)
            stations.append([i, input_data["Station"][i], input_data["Offset"][i]])

    for i in range(len(stations)):
        tolerance = tolerance_prepare(i, stations)
        # print(tolerance)
        s_class = make_class(input_data, stations[i][0], tolerance)
        classes.append(s_class)

    need_check = True
    dev = 1
    while need_check:
        dev += 1
        need_check = False
        classes_copy = deepcopy(classes)
        station_copy = deepcopy(stations)
        for station_i in tqdm(range(len(station_copy))):
            if len(classes_copy[station_i]) > number_of_each_station:
                start_temp, end_temp = classes_copy[station_i][0], classes_copy[station_i][-2]+1 # the last number is station_id
                print(start_temp, end_temp)
                raise
                for i in range(start_temp, end_temp):
                    if abs(input_data["Offset"][i]) <= station_offset_allowable*dev:
                        stations = add_between(stations, station_i, [i, input_data["Station"][i], input_data["Offset"][i]])
                        tolerance = tolerance_prepare(station_i+1, stations, dev=dev)
                        s_class = make_class(input_data, stations[station_i+1][0], tolerance)
                        # print(s_class)
                        classes = add_between(classes, station_i, s_class)
                classes.pop(station_i)
                stations.pop(station_i)
                need_check = True
    classes = sortit(classes)

    return classes

def correct_offset(input_data, data, station_id):
    station_metric = input_data["Station"][station_id]
    if station_metric%10 == 0:
        pass
    elif int(station_metric%10) <= 1:
        station_metric = int(str(station_metric)[:-3]) 
    if int(station_metric%10) >= 9:
        station_metric = int(str(station_metric)[:-3])+1
    station_new = [station_metric for _ in range(len(data))]
    offset = list(input_data.loc[data]["Offset"])
    index_ = data.index(station_id)
    # print(offset)
    offset = add_between(offset, index_, 0, remove_original=True)
    # print(offset)
    elevation = input_data.loc[data]["Elevation"]
    merged_data = list(zip(station_new, data, offset, elevation))
    return sorted(merged_data, key= lambda x: x[2])

def final_preparation(final_classes):
    final_result = []
    number_of_chain = 0
    for list_ in final_classes:
        number_of_chain += 1
        final_result.append(["chainage", list_[0][0]])
        for item in list_:
            final_result.append([item[2], item[3]])
    print("The number of stations is: ", number_of_chain)
    result = pd.DataFrame(final_result)
    return result

def export_data(ad1, ad2, ad3, result, out_of_range_index):
    result.to_csv(ad1, index=False, header=False)
    with open(ad2, "w") as file:
        for index, row in result.iterrows():
            file.write(f"{row[0]}\t{row[1]}\n")

    with open(ad3, "w") as file:
        for row in out_of_range_index:
            file.write(f"{row}\n")

def main(ad, station_offset_allowable, max_number_of_each):
    prepared_data, out_of_range_index = dataloader(ad)
    classes = special_sort(prepared_data, station_offset_allowable, max_number_of_each)
    corrected_classes = []
    for i in range(len(classes)):
        item = correct_offset(prepared_data, classes[i][:-1], int(classes[i][-1][:-3]))
        # if i == 2:
            # raise
        corrected_classes.append(item)
    result = final_preparation(corrected_classes)
    return result, out_of_range_index


address = "Raw-Data/D10 points.csv"
address_result_txt = "Raw-Data/result.txt"
address_result_excel = "Raw-Data/result.csv"
out_of_range_txt = "Raw-Data/out_of_ranges.txt"

station_offset_allowable = 0.1  # min offset that is assume to be zero.
max_number_of_each = 60  # how many nodes can be in one class at max

result, out_of_range_index = main(address, station_offset_allowable, max_number_of_each)
export_data(address_result_excel, address_result_txt, out_of_range_txt, result, out_of_range_index)



  0%|          | 0/586 [00:00<?, ?it/s]

5670 5730


RuntimeError: No active exception to reraise